In [ ]:
import os
import json
import pandas as pd
from tqdm import tqdm

In [ ]:
import os
import jsonlines
from tqdm import tqdm

save_names = ['llava_pretrain']
data = {name: [] for name in save_names}
file_path = '/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/alignment_2.5m.jsonl'
base_path = '/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment'

# Count the total number of lines for the progress bar
total_lines = sum(1 for _ in open(file_path))

with jsonlines.open(file_path) as reader:
    for entry in tqdm(reader, total=total_lines, desc="Processing entries"):
        data_split = entry['image'].split('/')[1]
        
        if data_split in save_names:
            if data_split == 'llava_pretrain':
                entry['image'] = entry['image'].replace('llava_pretrain/images', 'sbu558k')
            image_path = os.path.join(base_path, entry['image'])
            image_path = os.path.normpath(image_path)
            
            if os.path.exists(image_path):
                data[data_split].append(entry)
            else:
                break
# count the number of each dataset
for key, entries in data.items():
    print(f"{key}: {len(entries)}")


/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/sbu558k/00120/001200924.jpg

In [ ]:
# save as json for each data
for key, entries in data.items():
    save_path = f'/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/{key}.json'
    with open(save_path, 'w') as f:
        json.dump(entries, f)
    print(f"Saved {len(entries)} entries to {save_path}")

In [ ]:
import torch
from tqdm import tqdm
def get_linear_input(sequence_output):
    cls_token = sequence_output[:, 0]
    patch_tokens = sequence_output[:, 1:]
    linear_input = torch.cat([cls_token, patch_tokens.mean(dim=1)], dim=1)
    return linear_input
directory = '/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-base'
for filename in os.listdir(directory):
    if filename.endswith(".pt"):
        print(os.path.join(directory, filename))
        image_embedding = torch.load(os.path.join(directory, filename))
        image_embedding = get_linear_input(image_embedding)
        torch.save(image_embedding, os.path.join(directory, filename))
    else:
        continue

In [ ]:
import torch
import random

def compare_random_tensor_value(pt_file1, pt_file2, position=None):
    # 加载 .pt 文件
    tensor1 = torch.load(pt_file1,weights_only=True)
    tensor2 = torch.load(pt_file2,weights_only=True)
    # 如果没有指定位置，随机选择一个索引位置
    if position is None:
        position = random.randint(0, len(tensor1) - 1) 
    # 获取指定位置的值
    value1 = tensor1[position]
    value2 = tensor2[position]
    
    # 比较两个值并打印结果
 
    print(f"The values at position {position} of index {index} are different.\n"
              f"File 1 value: {value1}\nFile 2 value: {value2}")
    
    # return value1, value2

index = random.randint(150,200)
pt_file1 = f"/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/dreamclipcc3m_raw_caption/{index}.pt"
pt_file2 = f"/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/dreamclipcc3m_raw/{index}.pt"
compare_random_tensor_value(pt_file1,pt_file2)

In [ ]:
random.randint(120)

In [ ]:
import glob
import os
from natsort import natsorted
path_list = ['/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-large/ALLaVAVFLAN', '/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-large/LLaVA558K']
all_files = []
for dir_path in path_list:
    files = glob.glob(os.path.join(dir_path, "*.pt"))
    sorted_files = natsorted(files)
    all_files.extend(sorted_files)
all_files

In [ ]:
import os
import json
# read json
with open('/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/llava_pretrain.json', 'r') as f:
    sharegpt4v = json.load(f)
with open('/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/sbu558k.json', 'r') as f:
    sbu558k = json.load(f)
with open('/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/coco.json', 'r') as f:
    coco = json.load(f)




In [ ]:
# function to caculate avg length 
def avg_length(data):
    avg = 0
    for entry in data:
        avg += len(entry['conversations'][-1]['value'])
    return avg/len(data)

avg_length(sharegpt4v), avg_length(sbu558k), avg_length(coco)

In [ ]:
sharegpt4v[0]

In [ ]:
import os
import torch
from natsort import natsorted
import glob

def merge_pt_files(directory):
    # 获取所有的 .pt 文件路径，并按名字顺序排序
    files = glob.glob(os.path.join(directory, "*.pt"))
    files = natsorted(files)
    
    # 确保文件数量为偶数
    if len(files) % 2 != 0:
        raise ValueError("The number of .pt files is not even. Please make sure the files can be paired.")

    # 遍历所有文件，按两个为一组进行合并
    for i in range(0, len(files), 2):
        file1 = files[i]
        file2 = files[i+1]
        
        # 加载两个文件
        tensor1 = torch.load(file1)
        tensor2 = torch.load(file2)
        
        # 在第一个维度进行拼接
        merged_tensor = torch.cat((tensor1, tensor2), dim=0)
        
        # 生成新文件名
        base_name1 = os.path.basename(file1).replace(".pt", "")
        base_name2 = os.path.basename(file2).replace(".pt", "")
        new_file_name = f"{base_name1}_{base_name2}.pt"
        new_file_path = os.path.join(directory, new_file_name)
        
        # 保存拼接后的tensor
        torch.save(merged_tensor, new_file_path)
        print(f"Saved merged tensor to {new_file_path}")

# 示例用法
# merge_pt_files('/path/to/your/directory')


In [ ]:
import os
import glob
import torch
from tqdm import tqdm
file_dir = "/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/laion30m_caption"
files = glob.glob(os.path.join(file_dir, "*.pt"))
total_len = 0
for file in tqdm(files):
    tensor = torch.load(file, map_location=torch.device('cpu'), weights_only=False)
    total_len += tensor.size(0)
    if tensor.size(0) != 4096:
        print(file, tensor.size(0))
print(total_len)



In [1]:
import os
import glob
import torch
from tqdm import tqdm
file_dir = "/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/laion30m"
files = glob.glob(os.path.join(file_dir, "*.pt"))
total_len = 0
for file in tqdm(files):
    tensor = torch.load(file, map_location=torch.device('cpu'), weights_only=False)
    total_len += tensor.size(0)
    if tensor.size(0) != 4096:
        print(file, tensor.size(0))
print(total_len)



 52%|█████▏    | 2707/5170 [01:09<01:02, 39.31it/s]

/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/laion30m/5169.pt 642


100%|██████████| 5170/5170 [02:17<00:00, 37.59it/s]

21172866


In [ ]:
import torch
vision_embedding = torch.load('/home/mila/l/le.zhang/scratch/light_align/data/text_embedding/gte-large-en-v1.5/dreamclipcc12m_longSV_captions/5859.pt')
print(vision_embedding.shape)   
vision_embedding[54]

In [ ]:
import os
import glob
import torch
from tqdm import tqdm
file_dir = "/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-base/dreamclipcc12mhf"
files = glob.glob(os.path.join(file_dir, "*.pt"))
total_len = 0
for file in tqdm(files):
    tensor = torch.load(file, map_location=torch.device('cpu'), weights_only=False)
    total_len += tensor.size(0)
    if tensor.size(0) != 1024:
        print(file, tensor.size(0))
print(total_len)



In [ ]:
import os
import glob
import torch
from tqdm import tqdm
file_dir = "/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-giant/dreamclipcc3m"
files = glob.glob(os.path.join(file_dir, "*.pt"))
total_len = 0
for file in tqdm(files):
    tensor = torch.load(file, map_location=torch.device('cpu'), weights_only=False)
    total_len += tensor.size(0)
    if tensor.size(0) != 1024:
        print(file, tensor.size(0))
print(total_len)



In [ ]:
import pandas as pd
from data_config import DATADIR
# load csv file
csv = pd.read_csv(DATADIR['dreamclipcc12mhf']['annotation'], nrows=100000)
csv

In [ ]:
# open image and show
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
# 指定index为5的行是我们要显示的图像
index = random.randint(0, 100000)
image_path = os.path.join(DATADIR['dreamclipcc12mhf']['imagedir'], csv.iloc[index]['Image Path'])

# 打开图像并显示 标题为caption
image = Image.open(image_path)
plt.imshow(image)
plt.title("\n".join(csv.iloc[index]['longLLA_captions'].split(" ")), loc='left')
plt.show()


# 3: test if embedding are encoded correctly

In [2]:
import pandas as pd
import random
import os
from data_config import DATADIR
# csv = pd.read_csv('/home/mila/l/le.zhang/scratch/datasets/LAION/30M_laion_synthetic_filtered_large_with_path_filtered.csv')
csv = pd.read_csv(DATADIR['laion30m']['annotation'])
csv

,caption,url,Image Path
0,the tiki ball logo is displayed on this apron,https://image.spreadshirtmedia.com/image-serve...,LAION30M/images/0000000/0000000.jpg
1,the view of an aerial pool with palm trees,http://uberflip.cdntwrk.com/mediaproxy?url=htt...,LAION30M/images/0000000/0000001.jpg
2,a sky with clouds women's v - neck,https://render.fineartamerica.com/images/rende...,LAION30M/images/0000000/0000002.jpg
3,the brain coffee mug,https://cdn.shopify.com/s/files/1/2438/3835/pr...,LAION30M/images/0000000/0000003.jpg
4,the blue sedan is parked on a street,https://http2.mlstatic.com/ford-focus-20-st-5-...,LAION30M/images/0000000/0000006.jpg
...,...,...,...
21172861,the peace revolution episode 9 of reclaimivism,http://4.bp.blogspot.com/-q_984YJqDOw/TnDihomT...,LAION30M/images/0002999/0009995.jpg
21172862,2 pack - body - multi,https://img01.ztat.net/article/spp-media-p1/f3...,LAION30M/images/0002999/0009996.jpg
21172863,a large crystal chandelier hanging from the ce...,http://img1.wfrcdn.com/lf/50/hash/2023/866064/...,LAION30M/images/0002999/0009997.jpg
21172864,a white bed with white pillows,https://akamai-scene7.ballarddesigns.com/is/im...,LAION30M/images/0002999/0009998.jpg


### 3.1 compare text local embedding and online embedding

In [127]:
import torch
import random

import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

model_path = 'Alibaba-NLP/gte-large-en-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_path)
text_model = AutoModel.from_pretrained(model_path, trust_remote_code=True, torch_dtype=torch.float16).to('cuda')
text_model.eval()

NewModel(
  (embeddings): NewEmbeddings(
    (word_embeddings): Embedding(30528, 1024, padding_idx=0)
    (rotary_emb): NTKScalingRotaryEmbedding()
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): NewEncoder(
    (layer): ModuleList(
      (0-23): 24 x NewLayer(
        (attention): NewSdpaAttention(
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (mlp): NewGatedMLP(
          (up_gate_proj): Linear(in_features=1024, out_features=8192, bias=False)
          (down_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (act_fn): GELUActivation()
          (hidden_dropout): Dropout(p=0.1, inplace=False)
        )
        (attn_ln): LayerNorm((1024,), eps=1e-12, elementwis

In [118]:
# Requires transformers>=4.36.0

import torch.nn.functional as F
import random

def compare_embeddings(n_tests=20, verbose=False):
    def compute_difference(precomputed, computed):
        return F.mse_loss(precomputed, computed).item()

    differences = []
    all_test_index = []
    for _ in range(n_tests):
        file_idx = random.randint(0, 5000)
        idx = random.randint(0, 4096)

        # Load pre-computed embeddings
        x = torch.load(f'/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/3mvalid/{file_idx}.pt', weights_only=True)

        # Get the caption and its pre-computed embedding
        caption = csv.iloc[file_idx*4096+idx]['caption']
        precomputed_embedding = x[idx]

        # Compute the embedding on-the-fly for comparison
        with torch.no_grad():
            inputs = tokenizer(caption, max_length=8192, padding=True, truncation=True, return_tensors='pt').to('cuda')
            outputs = text_model(**inputs)
            computed_embedding = outputs.last_hidden_state[:, 0].to(torch.float16)[0].cpu()

        # Compute the difference
        diff = compute_difference(precomputed_embedding, computed_embedding)
        differences.append(diff)
        all_test_index.append(file_idx*4096+idx)
        if verbose:
            print(f"Test {_ + 1}:")
            print(f"Caption: {caption} from index {file_idx*4096+idx}")
            print(f"Precomputed embedding : {precomputed_embedding}")
            print(f"Computed embedding : {computed_embedding}")
            print(f"Difference (MSE): {diff}\n")

    # Print summary statistics
    print(f"All test index: {all_test_index}")
    print(f"Average difference over {n_tests} tests: {sum(differences) / n_tests}")
    print(f"Max difference: {max(differences)}")
    print(f"Min difference: {min(differences)}")

# Usage example:
compare_embeddings(n_tests=1000, verbose=False)


All test index: [11224793, 2314150, 17755481, 7629665, 12830764, 3611940, 19842386, 9227036, 9298819, 17344, 1941918, 9673912, 6833064, 17001136, 323185, 4582732, 7825298, 18901270, 6991378, 15200084, 16075713, 14582865, 16122920, 15649797, 1311608, 7053571, 18385141, 16339891, 1208540, 12412815, 14391713, 1373392, 13999264, 16903562, 9304337, 16575195, 994195, 13436781, 5186432, 12590612, 17034015, 12176162, 5945395, 17714559, 20422945, 15467545, 289533, 11976424, 12602272, 9449434, 13833732, 13190228, 15619484, 18901216, 16840990, 4628505, 2685743, 2439412, 6117084, 10234059, 13595409, 8452952, 216992, 5282152, 2945796, 265070, 18034370, 1220369, 15331761, 1369652, 11632682, 11518093, 11024603, 1401945, 13261722, 14345905, 20382069, 14018851, 2795945, 1632947, 12142869, 5037332, 17264291, 18749304, 1902694, 14062173, 15890297, 18762979, 10690189, 16205159, 1607592, 20104139, 4011394, 12588542, 11019574, 8994017, 9634936, 8913977, 18426912, 17572725, 5906248, 3244706, 7946656, 1080603

### 3.2 Test local image embedding and online computed embedding

In [3]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests
import os
import random
import torch.nn.functional as F
import time

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-large')
model = AutoModel.from_pretrained('facebook/dinov2-large',attn_implementation="sdpa", torch_dtype=torch.float16)
# model = AutoModel.from_pretrained('facebook/dinov2-large')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# model = torch.compile(model)
model.eval()

Dinov2Model(
  (embeddings): Dinov2Embeddings(
    (patch_embeddings): Dinov2PatchEmbeddings(
      (projection): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2Encoder(
    (layer): ModuleList(
      (0-23): 24 x Dinov2Layer(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2SdpaAttention(
          (attention): Dinov2SdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): Dinov2SelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (layer_scale1): Dinov2LayerScale()
      

In [7]:
def compare_image_embeddings(model,processor,n_tests=5, verbose=False):
    differences = []
    all_test_indices = []
    device = model.device

    for idx in range(5169):
        # file_idx = random.randint(0, 5169)
        file_idx = idx
        idx = random.randint(0, 4096)  # Changed to 0-4095 to match the tensor size
        test_index = file_idx * 4096 + idx
        all_test_indices.append(test_index)

        image_path = os.path.join(DATADIR['laion30m']['imagedir'], csv.iloc[test_index]['Image Path'])
        image = Image.open(image_path)
        
        
        if verbose:
            print(f"Test {_ + 1}:")
            print(f"Comparing image at index {test_index}")

        # Load pre-computed embedding
        emebdding_path = f"/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/laion30m/{file_idx}.pt"
        if not os.path.exists(emebdding_path):
            continue
        x = torch.load(emebdding_path, weights_only=True)
        precomputed_embedding = x[idx].to(device)

        # Compute embedding on-the-fly
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        sequence_output = outputs[0]
        cls_token = sequence_output[:, 0]
        patch_tokens = sequence_output[:, 1:]
        computed_embedding = torch.cat([cls_token, patch_tokens.mean(dim=1)], dim=1).to(torch.float16)[0]

        # Move embeddings back to CPU for comparison and storage
        precomputed_embedding = precomputed_embedding.cpu()
        computed_embedding = computed_embedding.cpu()

        if verbose:
            print(f"Precomputed embedding: {precomputed_embedding}")
            print(f"Computed embedding: {computed_embedding}")

        # Compute the difference
        diff = F.mse_loss(precomputed_embedding, computed_embedding).item()
        differences.append(diff)
        if diff>1:
            # os.remove(emebdding_path)
            print(f"remove {emebdding_path}, with diff {diff}")
            
        if verbose:
            print(f"Difference (MSE): {diff}\n")

    # Print summary statistics
    print(f"All test indices: {all_test_indices}")
    print(f"Average difference over {n_tests} tests: {sum(differences) / n_tests}")
    print(f"Max difference: {max(differences)}")
    print(f"Min difference: {min(differences)}")

start_time = time.time()
# Usage example:
compare_image_embeddings(model,processor,n_tests=1000, verbose=False)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

/home/mila/l/le.zhang/.conda/envs/openflamingo/lib/python3.9/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


### 3.2.1 Manually inspect single file

In [111]:
from PIL import Image
import torch

device = 'cuda'
file_idx = random.randint(0, 5000)
idx = random.randint(0, 4096)  # Changed to 0-4095 to match the tensor size
test_index = file_idx * 4096 + idx

image_path = os.path.join(DATADIR['laion30m']['imagedir'], csv.iloc[test_index]['Image Path'])
image = Image.open(image_path)


# Load pre-computed embedding
emebdding_path = f"/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/laion30m/{file_idx}.pt"

x = torch.load(emebdding_path, weights_only=True)
precomputed_embedding = x[idx].to(device)

# Compute embedding on-the-fly
inputs = processor(images=image, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)
sequence_output = outputs[0]
cls_token = sequence_output[:, 0]
patch_tokens = sequence_output[:, 1:]
computed_embedding = torch.cat([cls_token, patch_tokens.mean(dim=1)], dim=1).to(torch.float16)[0]

# Move embeddings back to CPU for comparison and storage
precomputed_embedding = precomputed_embedding.cpu()
computed_embedding = computed_embedding.cpu()
print(f"image index {test_index}")
print(f"Precomputed embedding: {precomputed_embedding}")
print(f"Computed embedding: {computed_embedding}")

FileNotFoundError: [Errno 2] No such file or directory: '/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/laion30m/1272.pt'

### 3.3 Visualize the image and its caption

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random
# Get the caption and image path
idx = random.randint(0, len(df))
caption = df.iloc[idx]['caption']
image_path = os.path.join("/home/mila/l/le.zhang/scratch/datasets/LAION", df.iloc[idx]['Image Path'])

# Open and display the image
img = Image.open(image_path)
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')

# Add the caption at the bottom of the image
plt.text(0.5, -0.05, caption, wrap=True, horizontalalignment='center', 
         verticalalignment='top', transform=plt.gca().transAxes)

plt.show()



In [ ]:
import torch 
import random
x1 = torch.load('/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/dreamclipcc3m/0.pt')
x2 = torch.load('/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/dreamclipcc3m1/0.pt').to(torch.float16)

In [ ]:
idx = random.randint(0, 1024)
x1[idx],x2[idx]

In [130]:
# randomly select 10000 samples from /home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/dreamclipcc3m and copy to /home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/validation, at
import os
import random
import shutil

image_source_dir = '/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/laion30m'
image_destination_dir = '/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/validation'
text_source_dir = '/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/laion30m_caption'
text_destination_dir = '/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/validation'

# Create the destination directory if it doesn't exist
os.makedirs(image_destination_dir, exist_ok=True)
os.makedirs(text_destination_dir, exist_ok=True)


file_index = random.sample(range(len(files)), 4)
print(file_index)
# copy image files
for file in file_index:
    shutil.copy(os.path.join(image_source_dir, str(file)+'.pt'), image_destination_dir)

# copy text files
for file in file_index:
    shutil.copy(os.path.join(text_source_dir, str(file)+'.pt'), text_destination_dir)



[1955, 406, 1164, 881]


In [1]:
import torch

In [9]:
x = torch.tensor([[1,2,3],[4,5,6],[7,8,9]],dtype=torch.float32)
torch.diagonal(x)

tensor([1., 5., 9.])

In [10]:
x

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

In [11]:
torch.mean(torch.diagonal(x))

tensor(5.)